# Snakemake版本的HiCPro构建

In [44]:
import glob
import json

## 参数设置

In [45]:
thread = 20

bowtie2_index
```shell
bowtie2-build -f genome_ucsc_mm39.fa genome_ucsc_mm39.fa.bowtie2_index --threads 24
```

chromosome_size
```shell
# hg38
samtools faidx genome_ucsc_hg38.fa
cut -f1-2 genome_ucsc_hg38.fa.fai > chrom_hg38.sizes

# mm39
samtools faidx genome_ucsc_mm39.fa
cut -f1-2 genome_ucsc_mm39.fa.fai > chrom_mm39.sizes
```

In [46]:
# hg38
# bowtie2_index = "/lustre1/chengqiyi_pkuhpc/zhaohn/1.database/db_genomes/genome_fa/genome_ucsc_hg38/genome_ucsc_hg38.fa.bowtie2_index"
# chr_sizes = "program/HiC-Pro_3.1.0/annotation/chrom_hg38.sizes"
# mm39
bowtie2_index = "/lustre1/chengqiyi_pkuhpc/zhaohn/1.database/db_genomes/genome_fa/genome_ucsc_mm39/genome_ucsc_mm39.fa.bowtie2_index"
chr_sizes = "program/HiC-Pro_3.1.0/annotation/chrom_mm39.sizes"

In [47]:
# bowtie2 options for mapping step1.
# 比对时采用--reorder参数，确保以fastq完全相同的顺序输出sam文件
# Default:
#     "--very-sensitive -L 30 --score-min L,-0.6,-0.2 --end-to-end --reorder"
bowtie2_params_global = (
    "--very-sensitive -L 30 --score-min L,-0.6,-0.2 --end-to-end --reorder"
)

# bowtie2 options for mapping step2.
# 比对时采用--reorder参数，确保以fastq完全相同的顺序输出sam文件
# Default:
#     "--very-sensitive -L 20 --score-min L,-0.6,-0.2 --end-to-end --reorder"
# 采用部分比对参数(--local)，以代替端到端比对(--end-to-end), 因为HiC是嵌合片段，因此使用部分比对提高比对效率
# 但是这里仍然要采用endtoend模式
# 原因详见 https://groups.google.com/g/hic-pro/c/2K43gMov7FI
# 这里还叫local是历史原因，我后面考虑去掉
bowtie2_params_local = (
    "--very-sensitive -L 20 --score-min L,-0.6,-0.2 --end-to-end --reorder"
)

In [48]:
# Ligation site sequence used for reads trimming.
# Depends on the fill in strategy.
# Example: AAGCTAGCTT
# 酶的序列，例如：
# HindIII，为AAGCTAGCTT
# MboI，为GATCGATC
ligation_site = "AAGCTAGCTT"

In [49]:
# samtools sort
#   -m INT     Set maximum memory per thread; suffix K/M/G recognized [768M]
# sort_ram_per_thread = "" # use default setting of samtools sort
sort_ram_per_thread = ""
# sort_ram_per_thread = "-m 500M"

In [50]:
# form_pairs_bam
# mergeSAM.py
# Minimum mapping quality. Reads with lower quality are discarded. Default: 0
min_mapq = 10

In [51]:
# form matrix of different bin size

#######################################################################
## Contact Maps
#######################################################################
# Resolution of contact maps to generate
# Default: 20_000 40_000 150_000 500_000 1_000_000

# For compartment: 10mb~100mb
# For TAD: 100kb~10mb
# For functional loops: 10kb~100kb
# For nucleosome clutches: 1kb~2kb

#######################################################################
bin_sizes = [
    # For nucleosome clutches: 1kb~2kb
    1_000,
    2_000,
    5_000,
    # For functional loops: 10kb~100kb
    10_000,
    20_000,
    50_000,
    100_000,
    # For TAD: 100kb~10mb
    200_000,
    500_000,
    1_000_000,
    # For compartment: 10mb~100mb
    10_000_000,
    50_000_000,
    100_000_000,
]

## 生成samples.json

In [52]:
ls = glob.glob("../fastq/*.fastq.gz")
ls.sort()
assert ls != []  # 需要非空
# 默认 HiC是双端测序
ls_pe = [i for i in ls if i.endswith("R1.fastq.gz")]

if ls_pe:
    ls_sample = [i.split("/")[-1].split("_R1.fastq")[0] for i in ls_pe]
    end_type = "PE"
ls_sample

['Hi-C_auxin-2days_rep1',
 'Hi-C_auxin-2days_rep2',
 'Hi-C_untreated_rep1',
 'Hi-C_untreated_rep2',
 'Hi-C_washoff-2days_rep1',
 'Hi-C_washoff-2days_rep2',
 'SRR400264_00',
 'SRR400264_01']

In [53]:
# for test and select
# ls_sample = [
# "Hi-C_auxin-2days_rep1",
# "Hi-C_auxin-2days_rep2",
# "Hi-C_untreated_rep1",
# "Hi-C_untreated_rep2",
# "Hi-C_washoff-2days_rep1",
# "Hi-C_washoff-2days_rep2",
# "SRR400264_00",  # for test
# "SRR400264_01",  # for test
# ]

In [54]:
dt = {
    "seq_mode": end_type,
    "samples": ls_sample,
    "thread": thread,
    "bowtie2_index": bowtie2_index,
    "bowtie2_params_global": bowtie2_params_global,
    "bowtie2_params_local": bowtie2_params_local,
    "ligation_site": ligation_site,
    "sort_ram_per_thread": sort_ram_per_thread,
    "min_mapq": min_mapq,
    "bin_sizes": bin_sizes,
    "chr_sizes": chr_sizes,
}
dt

{'seq_mode': 'PE',
 'samples': ['Hi-C_auxin-2days_rep1',
  'Hi-C_auxin-2days_rep2',
  'Hi-C_untreated_rep1',
  'Hi-C_untreated_rep2',
  'Hi-C_washoff-2days_rep1',
  'Hi-C_washoff-2days_rep2',
  'SRR400264_00',
  'SRR400264_01'],
 'thread': 20,
 'bowtie2_index': '/lustre1/chengqiyi_pkuhpc/zhaohn/1.database/db_genomes/genome_fa/genome_ucsc_mm39/genome_ucsc_mm39.fa.bowtie2_index',
 'bowtie2_params_global': '--very-sensitive -L 30 --score-min L,-0.6,-0.2 --end-to-end --reorder',
 'bowtie2_params_local': '--very-sensitive -L 20 --score-min L,-0.6,-0.2 --end-to-end --reorder',
 'ligation_site': 'AAGCTAGCTT',
 'sort_ram_per_thread': '',
 'min_mapq': 10,
 'bin_sizes': [1000,
  2000,
  5000,
  10000,
  20000,
  50000,
  100000,
  200000,
  500000,
  1000000,
  10000000,
  50000000,
  100000000],
 'chr_sizes': 'program/HiC-Pro_3.1.0/annotation/chrom_mm39.sizes'}

In [55]:
with open("./samples.json", "wt") as f:
    f.write(json.dumps(dt))